# Introduction
Top 100 songs of the year 2006 - 2020

## Project description

Here is supposed to be project description

## Importing libraries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import wget
import glob
import os
from twython import Twython
from credentials import * 
import requests
import json

# Downloading data and scraping HTML websites 
Downloading HTML webpages with top 100 songs by year from www.billboard.com
- url - https://www.billboard.com/charts/year-end/hot-100-songs

## Downloading data

Here download data

## Scraping
- Scrape data from downloaded webpages and transform it into a `dataframe`

In [2]:
def get_webpage_data(start_year, end_year):
    
    df_full = pd.DataFrame()
    
    for i in range(start_year,end_year+1):
        year = i
    
        year = str(year)

        web_page = BeautifulSoup(open('../Top-songs-2006-2020/HTMLs/Top_100_'+year+'.html'), 'html.parser')

        all_titles = web_page.select('.ye-chart-item__title')
        all_artist = web_page.select('.ye-chart-item__artist')
        all_ranks = web_page.select('.ye-chart-item__rank')

        ranks_list = []
        for rank in all_ranks:
            ranks_list.append(rank.get_text().replace('\n', ''))

        titles_list = []
        for title in all_titles:
            titles_list.append(title.get_text().replace('\n', ''))

        artist_list = []
        for artist in all_artist:
            artist_list.append(artist.get_text().replace('\n', ''))

        year_list = []
        for year in web_page.select('.ye-chart-item__primary-row'):
            year_list.append(year['data-date'])

        df = pd.DataFrame(data = [ranks_list, titles_list, artist_list, year_list]).T
        df.columns = ['Rank', 'Title', 'Artist', 'Year']

        df_full = df_full.append(df)
        
    return df_full

In [27]:
full_df = get_webpage_data(2006,2020)
full_df.reset_index(drop = True, inplace = True)
full_df

Rank                  Title                             Artist  Year
0       1                Bad Day                      Daniel Powter  2006
1       2            Temperature                          Sean Paul  2006
2       3            Promiscuous  Nelly Furtado Featuring Timbaland  2006
3       4       You're Beautiful                        James Blunt  2006
4       5         Hips Don't Lie      Shakira Featuring Wyclef Jean  2006
...   ...                    ...                                ...   ...
1493   96  More Than My Hometown                      Morgan Wallen  2020
1494   97          Lovin' On You                         Luke Combs  2020
1495   98               Said Sum                       Moneybagg Yo  2020
1496   99                  Slide                H.E.R. Featuring YG  2020
1497  100           Walk Em Down   NLE Choppa Featuring Roddy Ricch  2020

[1498 rows x 4 columns]

# Spotify API

## Set up API

In [10]:
client_id = 'd4ba214546c0407b82d21cac4727e14f'
client_secret = '777e3d2c832942448d61ee11640e8f85'

In [13]:
auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(auth_url, {
    'grant_type':'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']
headers = {'Authorization': 'Bearer {token}'.format(token = access_token)}

## Next step

In [96]:
track_name = "Hips Don't Lie"
url = 'https://api.spotify.com/v1/search'

In [97]:
track = requests.get(url, headers = headers, params = {'q':track_name, 'type':'track'}).json()

In [98]:
# cia track id: 0mUyMawtxj1CJ76kn9gIZK istraukti reikia

In [99]:
track_name = track['tracks']['items'][0]['name']
track_id = track['tracks']['items'][0]['id']
track_duration_ms = track['tracks']['items'][0]['duration_ms']
track_explicit = track['tracks']['items'][0]['explicit']
track_popularity = track['tracks']['items'][0]['popularity']

In [100]:
track_name

"Hips Don't Lie (feat. Wyclef Jean)"

In [101]:
track_id

'3ZFTkvIE7kyPt6Nu3PEa7V'

### What to extract from spotify:
- release_date
- track id
- artist id
- name (of the artist / band)
- type (artist/band)
- duration (seconds)
- genres
- popularity?
- 

## rough notebook

In [ ]:
'''
audio_features=[]
for track_id in artist_track_id:
    audio_url= 'https://api.spotify.com/v1/audio-features/'+track_id
    res=requests.get(audio_url,headers=headers)
    audio_info=res.json()
    audio_features.append(audio_info)
'''

In [16]:
artist_name = 'Depeche Mode'
url = 'https://api.spotify.com/v1/search'

In [24]:
artist = requests.get(url, params = {'q':artist_name, 'type':'artist', 'market':'DK'}, headers = headers)

In [25]:
artist

<Response [200]>

In [26]:
url = 'https://api.spotify.com/v1/artists'
requests.get(url, headers = headers, params = {'ids':'762310PdDnwsDxAQxzQkfX'}).json()

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/762310PdDnwsDxAQxzQkfX'},
   'followers': {'href': None, 'total': 3929177},
   'genres': ['dance rock',
    'new romantic',
    'new wave',
    'permanent wave',
    'synthpop'],
   'href': 'https://api.spotify.com/v1/artists/762310PdDnwsDxAQxzQkfX',
   'id': '762310PdDnwsDxAQxzQkfX',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/2ec1d1c7a48df4244f0ba708eafd28b7afa6166b',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ce06360d3762def7812fc5137e4e79c1467858bc',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/b46a67a3085884321512a7a759b78c54ddb2bb31',
     'width': 160}],
   'name': 'Depeche Mode',
   'popularity': 77,
   'type': 'artist',
   'uri': 'spotify:artist:762310PdDnwsDxAQxzQkfX'}]}

# Notes
- add spotify API key and secret to different jupyter notebook and run it form here